# LSTM Crypto Predictor Using Closing Prices

In this notebook, we build and train a custom LSTM RNN that uses a 14 day window of Bitcoin closing prices to predict the 15th day closing price. 

## Data Preparation

In this section, we prepare the training and testing data for the model. The model will use a rolling 14 day window to predict the 15th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:


   reshape((X_train.shape[0], X_train.shape[1], 1))


In [32]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [33]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [34]:
# Load the Training csv data for Bitcoin

df_data = pd.read_csv('../Resources/Training_data.csv', index_col=0, infer_datetime_format=True)
df_data.index.set_names('Date', inplace=True)

df_data['Target_ret'] = df_data.Returns.shift(-1)
df_data.dropna(inplace=True)
df_data['Buy_or_sell'] = df_data.Target_ret.apply(lambda x: 'Buy' if x > 0 else 'Dont_buy')
# df_data.columns.to_list()

In [35]:
curr_list = df_data.Currency.unique()
curr_list

array(['BTC/AUD', 'ETH/AUD', 'XRP/AUD', 'LTC/AUD', 'ADA/AUD', 'XLM/AUD',
       'BCH/AUD'], dtype=object)

### Take 1 currency at a time

In [36]:
currency = curr_list[0]
currency

'BTC/AUD'

In [37]:
df_curr_data = df_data.loc[df_data.Currency == currency].copy()
df_curr_data.sort_index(inplace=True)
df_curr_data.shape

(15353, 20)

In [38]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [39]:
# df_curr_data.columns

In [12]:
# Predict Closing Prices using a 14 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 15 and see how the model performance changes
window_size = 12

# Column index 3 is the 'Closing price' column
feature_column = 3
target_column = 3
X, y = window_data(df_curr_data, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[14919.86035156 14936.62011719 14957.4296875  14940.08007812
  14896.98046875 14921.33007812 14956.70019531 14912.26953125
  14882.90039062 14486.79003906 14486.19042969 14603.5703125 ]
 [14936.62011719 14957.4296875  14940.08007812 14896.98046875
  14921.33007812 14956.70019531 14912.26953125 14882.90039062
  14486.79003906 14486.19042969 14603.5703125  14615.11035156]
 [14957.4296875  14940.08007812 14896.98046875 14921.33007812
  14956.70019531 14912.26953125 14882.90039062 14486.79003906
  14486.19042969 14603.5703125  14615.11035156 14602.83007812]
 [14940.08007812 14896.98046875 14921.33007812 14956.70019531
  14912.26953125 14882.90039062 14486.79003906 14486.19042969
  14603.5703125  14615.11035156 14602.83007812 14589.1796875 ]
 [14896.98046875 14921.33007812 14956.70019531 14912.26953125
  14882.90039062 14486.79003906 14486.19042969 14603.5703125
  14615.11035156 14602.83007812 14589.1796875  14557.12011719]] 

y sample values:
[[14615.11035156]
 [14602.830

In [13]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [14]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Use the MinMaxScaler to scale data between 0 and 1.

# Scaling X data 
X_scaler = MinMaxScaler()
X_scaler = X_scaler.fit(X_train)

X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# Scaling y data 
y_scaler = MinMaxScaler()
y_scaler = y_scaler.fit(y_train)

y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

X_train[0]

array([0.36034579, 0.36111738, 0.35673912, 0.35595215, 0.35035517,
       0.34154183, 0.34308494, 0.34114654, 0.33986523, 0.31923115,
       0.31920526, 0.3242704 ])

In [15]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:1]} \n")
print (f"X_test sample values:\n{X_test[:1]}")

X_train sample values:
[[[0.36034579]
  [0.36111738]
  [0.35673912]
  [0.35595215]
  [0.35035517]
  [0.34154183]
  [0.34308494]
  [0.34114654]
  [0.33986523]
  [0.31923115]
  [0.31920526]
  [0.3242704 ]]] 

X_test sample values:
[[[0.99411782]
  [1.01495853]
  [0.9995728 ]
  [1.01039515]
  [1.02900514]
  [0.98194655]
  [0.97953912]
  [0.99058574]
  [1.01050265]
  [0.98308689]
  [1.0000081 ]
  [1.01307827]]]


## Build and Train the LSTM RNN

In this section, we designed a custom LSTM RNN and fit (train) it using the training data.

1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [17]:
X_train.shape[1]

12

In [18]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

model = Sequential()

number_units = window_size
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
# model.add(LSTM(number_units, input_shape=(X_train.shape[1], X_train.shape[2])))

model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [22]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['accuracy'])


In [23]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 12, 12)            672       
_________________________________________________________________
dropout (Dropout)            (None, 12, 12)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 12, 12)            1200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 12)                1200      
_________________________________________________________________
dropout_2 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1

In [25]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended

model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
10738/10738 [==============================] - 77s 7ms/step - loss: 2.5859e-04 - accuracy: 1.8625e-04
Epoch 2/10
10738/10738 [==============================] - 78s 7ms/step - loss: 0.0014 - accuracy: 1.8625e-04
Epoch 3/10
10738/10738 [==============================] - 81s 8ms/step - loss: 0.0012 - accuracy: 1.8625e-04
Epoch 4/10
10738/10738 [==============================] - 80s 7ms/step - loss: 0.0012 - accuracy: 1.8625e-04
Epoch 5/10
10738/10738 [==============================] - 80s 7ms/step - loss: 0.0012 - accuracy: 1.8625e-04
Epoch 6/10
10738/10738 [==============================] - 79s 7ms/step - loss: 0.0012 - accuracy: 1.8625e-04
Epoch 7/10
10738/10738 [==============================] - 76s 7ms/step - loss: 0.0013 - accuracy: 1.8625e-04
Epoch 8/10
10738/10738 [==============================] - 70s 7ms/step - loss: 0.0013 - accuracy: 1.8625e-04
Epoch 9/10
10738/10738 [==============================] - 67s 6ms/step - loss: 0.0013 - accuracy: 1.8625e-04
Epoch 10/10
107

## Model Performance

In this section, we will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [26]:
# Evaluate the model
model.evaluate(X_test, y_test)

144/144 [==============================] - 0s 2ms/step - loss: 1.8680 - accuracy: 0.0000e+00


[1.867997646331787, 0.0]

In [31]:
# Make some predictions
predicted = model.predict(X_test)
df_pred = pd.DataFrame(predicted)
df_pred.describe()

,0
count,4602.000000
mean,0.923738
std,0.000292
min,0.923232
25%,0.923440
50%,0.923757
75%,0.923991
max,0.924235


In [21]:
# Recover the original prices instead of the scaled version
predicted_prices = y_scaler.inverse_transform(predicted)
real_prices = y_scaler.inverse_transform(y_test.reshape(-1, 1))

In [22]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df_curr_data.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
Date,,
2021-03-04 19:00:00+00:00,62074.035156,65087.351562
2021-03-04 20:00:00+00:00,62297.503906,65087.671875
2021-03-04 21:00:00+00:00,62400.574219,65087.722656
2021-03-04 22:00:00+00:00,62106.011719,65087.714844
2021-03-04 23:00:00+00:00,62607.480469,65087.824219


In [23]:
# Plot the real vs predicted values as a line chart
stocks.hvplot( y=['Real', 'Predicted'], title='Predictor using the Returns', width=1200, height=700)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)